# Lean 6 - Mathlib4 : La Bibliotheque Mathematique

**Navigation** : [← Lean-5-Tactics](Lean-5-Tactics.ipynb) | [Index](Lean-1-Setup.ipynb) | [Lean-7-LLM-Integration →](Lean-7-LLM-Integration.ipynb)

---


## Introduction

Ce notebook explore **Mathlib4**, la bibliotheque mathematique communautaire pour Lean 4. Avec plus de **4 millions de lignes de code** formalise, **~8000 modules**, et plus de **100 contributeurs reguliers**, Mathlib est devenue la reference mondiale pour les mathematiques formelles.

### Pourquoi Mathlib est revolutionnaire

- **Verification formelle** : Chaque theoreme est prouve par machine, garantissant l'absence d'erreurs
- **Reutilisabilite** : Les preuves existantes servent de base pour de nouvelles decouvertes
- **Collaboration** : Des mathematiciens du monde entier contribuent
- **IA-ready** : Base d'entrainement pour les systemes comme AlphaProof et LeanCopilot

### Objectifs pedagogiques

1. Comprendre la structure et l'organisation de Mathlib4
2. Installer et configurer Mathlib dans un projet Lean
3. Maitriser les tactiques puissantes : `ring`, `linarith`, `omega`, `norm_num`
4. Naviguer dans la documentation et chercher des theoremes
5. Utiliser Mathlib pour des preuves non triviales

### Prerequis

- Notebooks **Lean-1** a **Lean-5** completes
- Connexion internet pour telecharger Mathlib

### Duree estimee : 45-50 minutes

---

## Contexte : L'Ecosysteme Mathlib

### Historique

| Annee | Evenement |
|-------|-----------|
| 2017 | Creation de Mathlib pour Lean 3 |
| 2023 | Migration complete vers Lean 4 (Mathlib4) |
| 2024 | Utilisation par AlphaProof (IMO medaille argent) |
| 2025 | Terry Tao formalise des resultats en theorie analytique |
| 2026 | Lean4Lean presente a POPL (bootstrap de Lean en Lean) |

### Etat actuel (Janvier 2026)

- **Version** : 4.27.0-rc1
- **Taille** : 4+ millions de lignes, ~8000 modules
- **Contributeurs** : 100+ reguliers, 500+ occasionnels
- **Utilisateurs notables** : Terry Tao (Fields Medal), Kevin Buzzard (Imperial)
- **Repository** : https://github.com/leanprover-community/mathlib4
- **Documentation** : https://leanprover-community.github.io/mathlib4_docs/

## 1. Installation et Configuration

### 1.1 Creer un projet avec Mathlib

Mathlib s'installe via **Lake**, le gestionnaire de paquets Lean 4.

### 1.1 Creer un projet avec Mathlib (Terminal)

Mathlib s'installe via **Lake**, le gestionnaire de paquets Lean 4. Voici les commandes a executer dans un terminal :

```bash
# Creer un nouveau projet avec Mathlib
lake new mon_projet math
cd mon_projet

# Telecharger les dependances
lake update

# Telecharger le cache compile (IMPORTANT - economise des heures!)
lake exe cache get
```

**Structure du projet resultant :**

```
mon_projet/
├── lakefile.lean      # Configuration Lake
├── lean-toolchain     # Version de Lean
├── MonProjet.lean     # Point d'entree
└── lake-manifest.json # Versions des dependances
```

**Note** : Le telechargement initial de Mathlib4 peut prendre 10-20 minutes et ~1 Go d'espace disque. Le cache precompile (`lake exe cache get`) evite de tout recompiler.

### 1.2 lakefile.lean type pour Mathlib

Le fichier `lakefile.lean` configure les dependances du projet. Pour Mathlib, il specifie la version et permet de telecharger le cache precompile avec `lake exe cache get`.

In [1]:
-- Exemple de lakefile.lean pour un projet Mathlib
/-
import Lake
open Lake DSL

package "mon_projet" where
  version := v!"0.1.0"

require mathlib from git
  "https://github.com/leanprover-community/mathlib4"

@[default_target]
lean_lib «MonProjet» where
  -- Configuration additionnelle si necessaire
-/

-- Exemple de lakefile.lean pour un projet Mathlib
/-
import Lake
open Lake DSL

package "mon_projet" where
  version := v!"0.1.0"

require mathlib from git
  "https://github.com/leanprover-community/mathlib4"

@[default_target]
lean_lib «MonProjet» where
  -- Configuration additionnelle si necessaire
-/
--% env 0

Raw input:
{"cmd": "-- Exemple de lakefile.lean pour un projet Mathlib\n/-\nimport Lake\nopen Lake DSL\n\npackage \"mon_projet\" where\n  version := v!\"0.1.0\"\n\nrequire mathlib from git\n  \"https://github.com/leanprover-community/mathlib4\"\n\n@[default_target]\nlean_lib \u00abMonProjet\u00bb where\n  -- Configuration additionnelle si necessaire\n-/"}
Raw output:
{"env": 0}

### 1.3 Importer Mathlib

Les imports Mathlib suivent une hierarchie : `Mathlib.Data.Nat.Prime` pour les nombres premiers, `Mathlib.Algebra.Ring.Basic` pour les anneaux, etc.

In [2]:
/-!
## Imports Mathlib typiques (a utiliser dans un projet avec Mathlib)

Dans un projet avec Mathlib installe, on utiliserait :
- `import Mathlib.Tactic` : Toutes les tactiques (ring, linarith, norm_num)
- `import Mathlib.Data.Nat.Prime` : Nombres premiers
- `import Mathlib.Algebra.Ring.Basic` : Structures d'anneaux
- `import Mathlib.Analysis.SpecialFunctions.Log.Basic` : Logarithmes

**Note importante** : Ce notebook s'execute dans un environnement Jupyter avec
lean4_jupyter, sans acces direct a Mathlib. Les exemples Mathlib sont documentes
en commentaires. Pour les executer, utilisez un projet Lake avec Mathlib.

Les tactiques disponibles dans Lean de base (sans Mathlib) incluent :
- `omega` : Arithmetique de Presburger (entiers avec +, -, <, <=)
- `simp` : Simplification automatique
- `rfl` : Reflexivite
- `decide` : Propositions decidables
-/

-- Verification que l'environnement Lean fonctionne
#check Nat
#check @Nat.add_comm

/-!
## Imports Mathlib typiques (a utiliser dans un projet avec Mathlib)

Dans un projet avec Mathlib installe, on utiliserait :
- `import Mathlib.Tactic` : Toutes les tactiques (ring, linarith, norm_num)
- `import Mathlib.Data.Nat.Prime` : Nombres premiers
- `import Mathlib.Algebra.Ring.Basic` : Structures d'anneaux
- `import Mathlib.Analysis.SpecialFunctions.Log.Basic` : Logarithmes

**Note importante** : Ce notebook s'execute dans un environnement Jupyter avec
lean4_jupyter, sans acces direct a Mathlib. Les exemples Mathlib sont documentes
en commentaires. Pour les executer, utilisez un projet Lake avec Mathlib.

Les tactiques disponibles dans Lean de base (sans Mathlib) incluent :
- `omega` : Arithmetique de Presburger (entiers avec +, -, <, <=)
- `simp` : Simplification automatique
- `rfl` : Reflexivite
- `decide` : Propositions decidables
-/

-- Verification que l'environnement Lean fonctionne
#check Nat
──────▶  Nat : Type
#check @Nat.add_comm
──────▶  Nat.add_comm : ∀ (n m : Nat), n + m = m + n
--% env 1

Raw input:
{"cmd": "/-!\n## Imports Mathlib typiques (a utiliser dans un projet avec Mathlib)\n\nDans un projet avec Mathlib installe, on utiliserait :\n- `import Mathlib.Tactic` : Toutes les tactiques (ring, linarith, norm_num)\n- `import Mathlib.Data.Nat.Prime` : Nombres premiers\n- `import Mathlib.Algebra.Ring.Basic` : Structures d'anneaux\n- `import Mathlib.Analysis.SpecialFunctions.Log.Basic` : Logarithmes\n\n**Note importante** : Ce notebook s'execute dans un environnement Jupyter avec\nlean4_jupyter, sans acces direct a Mathlib. Les exemples Mathlib sont documentes\nen commentaires. Pour les executer, utilisez un projet Lake avec Mathlib.\n\nLes tactiques disponibles dans Lean de base (sans Mathlib) incluent :\n- `omega` : Arithmetique de Presburger (entiers avec +, -, <, <=)\n- `simp` : Simplification automatique\n- `rfl` : Reflexivite\n- `decide` : Propositions decidables\n-/\n\n-- Verification que l'environnement Lean fonctionne\n#check Nat\n#check @Nat.add_comm", "env": 0}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 22, "column": 0},
   "endPos": {"line": 22, "column": 6},
   "data": "Nat : Type"},
  {"severity": "info",
   "pos": {"line": 23, "column": 0},
   "endPos": {"line": 23, "column": 6},
   "data": "Nat.add_comm : ∀ (n m : Nat), n + m = m + n"}],
 "env": 1}

## 2. Structure de Mathlib4

### 2.1 Organisation des modules

Mathlib est organise en namespaces thematiques :

| Namespace | Contenu | Exemples |
|-----------|---------|----------|
| `Mathlib.Data` | Structures de donnees | List, Set, Finset, Multiset |
| `Mathlib.Algebra` | Structures algebriques | Group, Ring, Field, Module |
| `Mathlib.Order` | Relations d'ordre | Lattice, Complete lattice |
| `Mathlib.Topology` | Topologie generale | Espaces topologiques, compacite |
| `Mathlib.Analysis` | Analyse reelle/complexe | Limites, derivees, integrales |
| `Mathlib.NumberTheory` | Theorie des nombres | Primalite, congruences, Legendre |
| `Mathlib.Logic` | Logique avancee | Logique classique, cardinalite |
| `Mathlib.Tactic` | Tactiques Mathlib | ring, linarith, omega, polyrith |

### 2.2 Hierarchie de structures algebriques

Mathlib definit une hierarchie riche de structures algebriques :

```
Semigroup -> Monoid -> Group
    |           |         |
    v           v         v
CommSemigroup -> CommMonoid -> CommGroup
    |           |         |
    v           v         v
Semiring  ->   Ring   ->  Field
```

Cette hierarchie utilise les **typeclasses** de Lean pour l'inference automatique.

In [3]:
-- Exemple conceptuel de typeclass (syntaxe simplifiee)
-- class Monoid (M : Type*) extends One M, Mul M where
--   one_mul : forall a : M, 1 * a = a
--   mul_one : forall a : M, a * 1 = a
--   mul_assoc : forall a b c : M, (a * b) * c = a * (b * c)

-- Nat est automatiquement un Monoid additif et multiplicatif
#check @Nat.add_comm      -- Commutativity de l'addition
#check @Nat.mul_assoc     -- Associativite de la multiplication

-- Exemple conceptuel de typeclass (syntaxe simplifiee)
-- class Monoid (M : Type*) extends One M, Mul M where
--   one_mul : forall a : M, 1 * a = a
--   mul_one : forall a : M, a * 1 = a
--   mul_assoc : forall a b c : M, (a * b) * c = a * (b * c)

-- Nat est automatiquement un Monoid additif et multiplicatif
#check @Nat.add_comm      -- Commutativity de l'addition
──────▶  Nat.add_comm : ∀ (n m : Nat), n + m = m + n
#check @Nat.mul_assoc     -- Associativite de la multiplication
──────▶  Nat.mul_assoc : ∀ (n m k : Nat), n * m * k = n * (m * k)
--% env 2

Raw input:
{"cmd": "-- Exemple conceptuel de typeclass (syntaxe simplifiee)\n-- class Monoid (M : Type*) extends One M, Mul M where\n--   one_mul : forall a : M, 1 * a = a\n--   mul_one : forall a : M, a * 1 = a\n--   mul_assoc : forall a b c : M, (a * b) * c = a * (b * c)\n\n-- Nat est automatiquement un Monoid additif et multiplicatif\n#check @Nat.add_comm      -- Commutativity de l'addition\n#check @Nat.mul_assoc     -- Associativite de la multiplication", "env": 1}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 8, "column": 0},
   "endPos": {"line": 8, "column": 6},
   "data": "Nat.add_comm : ∀ (n m : Nat), n + m = m + n"},
  {"severity": "info",
   "pos": {"line": 9, "column": 0},
   "endPos": {"line": 9, "column": 6},
   "data": "Nat.mul_assoc : ∀ (n m k : Nat), n * m * k = n * (m * k)"}],
 "env": 2}

## 3. Tactiques Mathlib Essentielles

Mathlib fournit des tactiques **puissantes** qui automatisent de nombreuses preuves. Ces tactiques ne sont pas dans Lean de base mais deviennent indispensables pour les mathematiques serieuses.

### 3.1 `ring` : Algebre polynomiale

La tactique `ring` resout automatiquement les egalites dans les anneaux (commutatifs ou non).

In [4]:
-- Exemples avec ring (necessite Mathlib.Tactic)
-- Ces exemples fonctionnent dans un projet avec Mathlib installe

-- Identite remarquable
-- example (a b : Int) : (a + b)^2 = a^2 + 2*a*b + b^2 := by ring

-- Factorisation
-- example (a b : Int) : a^2 - b^2 = (a + b) * (a - b) := by ring

-- Expression complexe
-- example (x y z : Int) : 
--   (x + y + z)^2 = x^2 + y^2 + z^2 + 2*x*y + 2*x*z + 2*y*z := by ring

-- Pour Nat (sans Mathlib), on peut utiliser des lemmes manuellement
-- Voici une version simplifiee demontrant le concept
theorem double_times (a : Nat) : a + a = 2 * a := by
  simp [Nat.two_mul]

-- Avec Mathlib installe, on prouverait directement :
-- theorem square_expand (a b : Nat) : (a + b) * (a + b) = a*a + 2*a*b + b*b := by ring

-- Exemples avec ring (necessite Mathlib.Tactic)
-- Ces exemples fonctionnent dans un projet avec Mathlib installe

-- Identite remarquable
-- example (a b : Int) : (a + b)^2 = a^2 + 2*a*b + b^2 := by ring

-- Factorisation
-- example (a b : Int) : a^2 - b^2 = (a + b) * (a - b) := by ring

-- Expression complexe
-- example (x y z : Int) : 
--   (x + y + z)^2 = x^2 + y^2 + z^2 + 2*x*y + 2*x*z + 2*y*z := by ring

-- Pour Nat (sans Mathlib), on peut utiliser des lemmes manuellement
-- Voici une version simplifiee demontrant le concept
theorem double_times (a : Nat) : a + a = 2 * a := by
  simp [Nat.two_mul]

-- Avec Mathlib installe, on prouverait directement :
-- theorem square_expand (a b : Nat) : (a + b) * (a + b) = a*a + 2*a*b + b*b := by ring
--% env 3

Raw input:
{"cmd": "-- Exemples avec ring (necessite Mathlib.Tactic)\n-- Ces exemples fonctionnent dans un projet avec Mathlib installe\n\n-- Identite remarquable\n-- example (a b : Int) : (a + b)^2 = a^2 + 2*a*b + b^2 := by ring\n\n-- Factorisation\n-- example (a b : Int) : a^2 - b^2 = (a + b) * (a - b) := by ring\n\n-- Expression complexe\n-- example (x y z : Int) : \n--   (x + y + z)^2 = x^2 + y^2 + z^2 + 2*x*y + 2*x*z + 2*y*z := by ring\n\n-- Pour Nat (sans Mathlib), on peut utiliser des lemmes manuellement\n-- Voici une version simplifiee demontrant le concept\ntheorem double_times (a : Nat) : a + a = 2 * a := by\n  simp [Nat.two_mul]\n\n-- Avec Mathlib installe, on prouverait directement :\n-- theorem square_expand (a b : Nat) : (a + b) * (a + b) = a*a + 2*a*b + b*b := by ring", "env": 2}
Raw output:
{"env": 3}

### 3.2 `linarith` : Arithmetique lineaire

La tactique `linarith` resout automatiquement les inegalites lineaires sur les nombres reels, rationnels ou entiers. Elle combine les hypotheses pour trouver une contradiction.

In [5]:
-- Exemples avec linarith (Mathlib.Tactic)

-- Inegalite simple
-- example (x : Int) (h : x > 5) : x >= 5 := by linarith

-- Combinaison d'hypotheses
-- example (x y : Int) (h1 : x <= y) (h2 : y <= x + 3) : y - x <= 3 := by linarith

-- Systeme d'inegalites
-- example (a b c : Int) (h1 : a + b > c) (h2 : b + c > a) (h3 : c + a > b) :
--   a + b + c > 0 := by linarith

-- Version avec omega (disponible dans Lean de base pour Nat/Int)
theorem ineq_example (x y : Nat) (h1 : x <= y) (h2 : y <= x + 3) : y - x <= 3 := by
  omega

-- Exemples avec linarith (Mathlib.Tactic)

-- Inegalite simple
-- example (x : Int) (h : x > 5) : x >= 5 := by linarith

-- Combinaison d'hypotheses
-- example (x y : Int) (h1 : x <= y) (h2 : y <= x + 3) : y - x <= 3 := by linarith

-- Systeme d'inegalites
-- example (a b c : Int) (h1 : a + b > c) (h2 : b + c > a) (h3 : c + a > b) :
--   a + b + c > 0 := by linarith

-- Version avec omega (disponible dans Lean de base pour Nat/Int)
theorem ineq_example (x y : Nat) (h1 : x <= y) (h2 : y <= x + 3) : y - x <= 3 := by
  omega
--% env 4

Raw input:
{"cmd": "-- Exemples avec linarith (Mathlib.Tactic)\n\n-- Inegalite simple\n-- example (x : Int) (h : x > 5) : x >= 5 := by linarith\n\n-- Combinaison d'hypotheses\n-- example (x y : Int) (h1 : x <= y) (h2 : y <= x + 3) : y - x <= 3 := by linarith\n\n-- Systeme d'inegalites\n-- example (a b c : Int) (h1 : a + b > c) (h2 : b + c > a) (h3 : c + a > b) :\n--   a + b + c > 0 := by linarith\n\n-- Version avec omega (disponible dans Lean de base pour Nat/Int)\ntheorem ineq_example (x y : Nat) (h1 : x <= y) (h2 : y <= x + 3) : y - x <= 3 := by\n  omega", "env": 3}
Raw output:
{"env": 4}

### 3.3 `omega` : Arithmetique Presburger

La tactique `omega` est un decideur complet pour l'arithmetique de Presburger (entiers avec +, -, comparaisons). Plus puissante que `linarith` pour les entiers.

In [6]:
-- omega est disponible dans Lean de base pour Nat et Int

-- Egalites et inegalites
theorem omega_demo1 (n m : Nat) : n + m = m + n := by omega

theorem omega_demo2 (n : Nat) (h : n > 0) : n >= 1 := by omega

-- Systemes complexes
theorem omega_complex (a b c : Int)
  (h1 : a + b = c) (h2 : a - b = 0) : 2 * a = c := by omega

-- Divisibilite (modulo)
-- theorem div_mod (n : Nat) : n = n / 2 * 2 + n % 2 := by omega

-- omega est disponible dans Lean de base pour Nat et Int

-- Egalites et inegalites
theorem omega_demo1 (n m : Nat) : n + m = m + n := by omega

theorem omega_demo2 (n : Nat) (h : n > 0) : n >= 1 := by omega

-- Systemes complexes
theorem omega_complex (a b c : Int)
  (h1 : a + b = c) (h2 : a - b = 0) : 2 * a = c := by omega

-- Divisibilite (modulo)
-- theorem div_mod (n : Nat) : n = n / 2 * 2 + n % 2 := by omega
--% env 5

Raw input:
{"cmd": "-- omega est disponible dans Lean de base pour Nat et Int\n\n-- Egalites et inegalites\ntheorem omega_demo1 (n m : Nat) : n + m = m + n := by omega\n\ntheorem omega_demo2 (n : Nat) (h : n > 0) : n >= 1 := by omega\n\n-- Systemes complexes\ntheorem omega_complex (a b c : Int)\n  (h1 : a + b = c) (h2 : a - b = 0) : 2 * a = c := by omega\n\n-- Divisibilite (modulo)\n-- theorem div_mod (n : Nat) : n = n / 2 * 2 + n % 2 := by omega", "env": 4}
Raw output:
{"env": 5}

### 3.4 `norm_num` : Calcul numerique

La tactique `norm_num` evalue les expressions numeriques et verifie les proprietes calculables (primalite, divisibilite, etc.). Indispensable pour les assertions concretes.

In [7]:
-- Exemples avec norm_num (Mathlib.Tactic.NormNum)

-- Calculs simples
-- example : (2 : Nat) + 2 = 4 := by norm_num
-- example : (123 : Nat) * 456 = 56088 := by norm_num

-- Primalite
-- example : Nat.Prime 17 := by norm_num
-- example : ¬ Nat.Prime 15 := by norm_num

-- Comparaisons
-- example : (100 : Nat) < 200 := by norm_num

-- Sans Mathlib, on utilise decide ou rfl
theorem calc_example : 2 + 2 = 4 := by rfl
theorem compare_example : 5 < 10 := by decide

-- Exemples avec norm_num (Mathlib.Tactic.NormNum)

-- Calculs simples
-- example : (2 : Nat) + 2 = 4 := by norm_num
-- example : (123 : Nat) * 456 = 56088 := by norm_num

-- Primalite
-- example : Nat.Prime 17 := by norm_num
-- example : ¬ Nat.Prime 15 := by norm_num

-- Comparaisons
-- example : (100 : Nat) < 200 := by norm_num

-- Sans Mathlib, on utilise decide ou rfl
theorem calc_example : 2 + 2 = 4 := by rfl
theorem compare_example : 5 < 10 := by decide
--% env 6

Raw input:
{"cmd": "-- Exemples avec norm_num (Mathlib.Tactic.NormNum)\n\n-- Calculs simples\n-- example : (2 : Nat) + 2 = 4 := by norm_num\n-- example : (123 : Nat) * 456 = 56088 := by norm_num\n\n-- Primalite\n-- example : Nat.Prime 17 := by norm_num\n-- example : \u00ac Nat.Prime 15 := by norm_num\n\n-- Comparaisons\n-- example : (100 : Nat) < 200 := by norm_num\n\n-- Sans Mathlib, on utilise decide ou rfl\ntheorem calc_example : 2 + 2 = 4 := by rfl\ntheorem compare_example : 5 < 10 := by decide", "env": 5}
Raw output:
{"env": 6}

### 3.5 `field_simp` : Simplification dans les corps

`field_simp` simplifie les expressions avec divisions et fractions.

In [8]:
/-!
## Exemples avec field_simp (Mathlib.Tactic.FieldSimp)

La tactique `field_simp` simplifie les expressions avec divisions et fractions.
Elle necessite Mathlib.

**Avec Mathlib installe :**
```lean
-- Fractions
example (a b : Rat) (hb : b ≠ 0) : a / b * b = a := by field_simp

-- Addition de fractions
example (a b c : Rat) (hb : b ≠ 0) (hc : c ≠ 0) :
  a / b + a / c = a * (b + c) / (b * c) := by field_simp; ring

-- Simplification complexe
example (x : Real) (hx : x ≠ 0) : (x + 1) / x - 1 / x = 1 := by
  field_simp
  ring
```
-/

-- Sans Mathlib, on peut travailler avec les rationnels de Lean de base
-- mais les simplifications de fractions sont plus limitees
#check @Rat.add_comm
#check @Rat.mul_comm

/-!
## Exemples avec field_simp (Mathlib.Tactic.FieldSimp)

La tactique `field_simp` simplifie les expressions avec divisions et fractions.
Elle necessite Mathlib.

**Avec Mathlib installe :**
```lean
-- Fractions
example (a b : Rat) (hb : b ≠ 0) : a / b * b = a := by field_simp

-- Addition de fractions
example (a b c : Rat) (hb : b ≠ 0) (hc : c ≠ 0) :
  a / b + a / c = a * (b + c) / (b * c) := by field_simp; ring

-- Simplification complexe
example (x : Real) (hx : x ≠ 0) : (x + 1) / x - 1 / x = 1 := by
  field_simp
  ring
```
-/

-- Sans Mathlib, on peut travailler avec les rationnels de Lean de base
-- mais les simplifications de fractions sont plus limitees
#check @Rat.add_comm
        ────────────▶ ❌ unknown identifier 'Rat.add_comm'
#check @Rat.mul_comm
        ────────────▶ ❌ unknown identifier 'Rat.mul_comm'
--% env 7

Raw input:
{"cmd": "/-!\n## Exemples avec field_simp (Mathlib.Tactic.FieldSimp)\n\nLa tactique `field_simp` simplifie les expressions avec divisions et fractions.\nElle necessite Mathlib.\n\n**Avec Mathlib installe :**\n```lean\n-- Fractions\nexample (a b : Rat) (hb : b \u2260 0) : a / b * b = a := by field_simp\n\n-- Addition de fractions\nexample (a b c : Rat) (hb : b \u2260 0) (hc : c \u2260 0) :\n  a / b + a / c = a * (b + c) / (b * c) := by field_simp; ring\n\n-- Simplification complexe\nexample (x : Real) (hx : x \u2260 0) : (x + 1) / x - 1 / x = 1 := by\n  field_simp\n  ring\n```\n-/\n\n-- Sans Mathlib, on peut travailler avec les rationnels de Lean de base\n-- mais les simplifications de fractions sont plus limitees\n#check @Rat.add_comm\n#check @Rat.mul_comm", "env": 6}
Raw output:
{"messages":
 [{"severity": "error",
   "pos": {"line": 25, "column": 8},
   "endPos": {"line": 25, "column": 20},
   "data": "unknown identifier 'Rat.add_comm'"},
  {"severity": "error",
   "pos": {"line": 26, "column": 8},
   "endPos": {"line": 26, "column": 20},
   "data": "unknown identifier 'Rat.mul_comm'"}],
 "env": 7}

### 3.6 `polyrith` : Certificats polynomiaux

`polyrith` trouve automatiquement des preuves polynomiales en interrogeant **Sage** (systeme de calcul formel).

In [9]:
/-!
## Exemples avec polyrith (Mathlib.Tactic.Polyrith)

La tactique `polyrith` trouve automatiquement des preuves polynomiales
en interrogeant **Sage** (systeme de calcul formel externe).

**Avec Mathlib installe et Sage disponible :**
```lean
-- Egalite polynomiale non triviale
example (a b : Int) (h1 : a + b = 4) (h2 : a - b = 2) : a = 3 := by polyrith

-- Systeme d'equations
example (x y : Rat) (h1 : 2*x + 3*y = 7) (h2 : x - y = 1) : 
  x = 2 := by polyrith
```
-/

-- Sans polyrith, on peut resoudre manuellement avec omega pour les entiers
theorem systeme_lineaire (a b : Int) (h1 : a + b = 4) (h2 : a - b = 2) : a = 3 := by
  omega

theorem systeme_lineaire2 (a b : Int) (h1 : a + b = 4) (h2 : a - b = 2) : b = 1 := by
  omega

/-!
## Exemples avec polyrith (Mathlib.Tactic.Polyrith)

La tactique `polyrith` trouve automatiquement des preuves polynomiales
en interrogeant **Sage** (systeme de calcul formel externe).

**Avec Mathlib installe et Sage disponible :**
```lean
-- Egalite polynomiale non triviale
example (a b : Int) (h1 : a + b = 4) (h2 : a - b = 2) : a = 3 := by polyrith

-- Systeme d'equations
example (x y : Rat) (h1 : 2*x + 3*y = 7) (h2 : x - y = 1) : 
  x = 2 := by polyrith
```
-/

-- Sans polyrith, on peut resoudre manuellement avec omega pour les entiers
theorem systeme_lineaire (a b : Int) (h1 : a + b = 4) (h2 : a - b = 2) : a = 3 := by
  omega

theorem systeme_lineaire2 (a b : Int) (h1 : a + b = 4) (h2 : a - b = 2) : b = 1 := by
  omega
--% env 8

Raw input:
{"cmd": "/-!\n## Exemples avec polyrith (Mathlib.Tactic.Polyrith)\n\nLa tactique `polyrith` trouve automatiquement des preuves polynomiales\nen interrogeant **Sage** (systeme de calcul formel externe).\n\n**Avec Mathlib installe et Sage disponible :**\n```lean\n-- Egalite polynomiale non triviale\nexample (a b : Int) (h1 : a + b = 4) (h2 : a - b = 2) : a = 3 := by polyrith\n\n-- Systeme d'equations\nexample (x y : Rat) (h1 : 2*x + 3*y = 7) (h2 : x - y = 1) : \n  x = 2 := by polyrith\n```\n-/\n\n-- Sans polyrith, on peut resoudre manuellement avec omega pour les entiers\ntheorem systeme_lineaire (a b : Int) (h1 : a + b = 4) (h2 : a - b = 2) : a = 3 := by\n  omega\n\ntheorem systeme_lineaire2 (a b : Int) (h1 : a + b = 4) (h2 : a - b = 2) : b = 1 := by\n  omega", "env": 7}
Raw output:
{"env": 8}

## 4. Exemples de Theoremes Mathlib

### 4.1 Theorie des nombres

In [10]:
/-!
## Theorie des nombres avec Mathlib

Avec `import Mathlib.Data.Nat.Prime`, on accede a :
- `Nat.Prime` : Definition de nombre premier
- `Nat.Prime.two_le` : p premier => p >= 2  
- `Nat.Prime.one_lt` : p premier => 1 < p
- `Nat.minFac_prime` : Plus petit facteur premier
- `Nat.exists_infinite_primes` : Theoreme d'Euclide (infinite de premiers)

**Exemple avec norm_num :**
```lean
theorem prime_17 : Nat.Prime 17 := by norm_num
```
-/

-- Sans Mathlib, on peut definir et utiliser des predicats simples sur les nombres
def estPair (n : Nat) : Prop := n % 2 = 0
def estImpair (n : Nat) : Prop := n % 2 = 1

-- Preuves basiques
theorem deux_pair : estPair 2 := by rfl
theorem trois_impair : estImpair 3 := by rfl
theorem seize_pair : estPair 16 := by rfl

-- Lemmes sur la parite avec omega
theorem pair_plus_pair (a b : Nat) (ha : estPair a) (hb : estPair b) : estPair (a + b) := by
  unfold estPair at *
  omega

/-!
## Theorie des nombres avec Mathlib

Avec `import Mathlib.Data.Nat.Prime`, on accede a :
- `Nat.Prime` : Definition de nombre premier
- `Nat.Prime.two_le` : p premier => p >= 2  
- `Nat.Prime.one_lt` : p premier => 1 < p
- `Nat.minFac_prime` : Plus petit facteur premier
- `Nat.exists_infinite_primes` : Theoreme d'Euclide (infinite de premiers)

**Exemple avec norm_num :**
```lean
theorem prime_17 : Nat.Prime 17 := by norm_num
```
-/

-- Sans Mathlib, on peut definir et utiliser des predicats simples sur les nombres
def estPair (n : Nat) : Prop := n % 2 = 0
def estImpair (n : Nat) : Prop := n % 2 = 1

-- Preuves basiques
theorem deux_pair : estPair 2 := by rfl
theorem trois_impair : estImpair 3 := by rfl
theorem seize_pair : estPair 16 := by rfl

-- Lemmes sur la parite avec omega
theorem pair_plus_pair (a b : Nat) (ha : estPair a) (hb : estPair b) : estPair (a + b) := by
  unfold estPair at *
  omega
--% env 9

Raw input:
{"cmd": "/-!\n## Theorie des nombres avec Mathlib\n\nAvec `import Mathlib.Data.Nat.Prime`, on accede a :\n- `Nat.Prime` : Definition de nombre premier\n- `Nat.Prime.two_le` : p premier => p >= 2  \n- `Nat.Prime.one_lt` : p premier => 1 < p\n- `Nat.minFac_prime` : Plus petit facteur premier\n- `Nat.exists_infinite_primes` : Theoreme d'Euclide (infinite de premiers)\n\n**Exemple avec norm_num :**\n```lean\ntheorem prime_17 : Nat.Prime 17 := by norm_num\n```\n-/\n\n-- Sans Mathlib, on peut definir et utiliser des predicats simples sur les nombres\ndef estPair (n : Nat) : Prop := n % 2 = 0\ndef estImpair (n : Nat) : Prop := n % 2 = 1\n\n-- Preuves basiques\ntheorem deux_pair : estPair 2 := by rfl\ntheorem trois_impair : estImpair 3 := by rfl\ntheorem seize_pair : estPair 16 := by rfl\n\n-- Lemmes sur la parite avec omega\ntheorem pair_plus_pair (a b : Nat) (ha : estPair a) (hb : estPair b) : estPair (a + b) := by\n  unfold estPair at *\n  omega", "env": 8}
Raw output:
{"env": 9}

### 4.2 Algebre

In [11]:
/-!
## Algebre avec Mathlib

Avec `import Mathlib.Algebra.Group.Basic`, on accede aux proprietes generiques :

**Groupes :**
- `mul_one` : a * 1 = a
- `one_mul` : 1 * a = a  
- `mul_assoc` : (a * b) * c = a * (b * c)
- `mul_inv_cancel` : a * a⁻¹ = 1

**Anneaux :**
- `add_mul` : (a + b) * c = a * c + b * c
- `mul_add` : a * (b + c) = a * b + a * c
- `neg_add_cancel` : -a + a = 0

Ces theoremes s'appliquent a TOUS les types avec la structure adequate
(Int, Rat, Real, Complex, matrices, polynomes, etc.).
-/

-- Sans Mathlib, on a les proprietes specifiques aux types concrets
-- Voici quelques exemples avec Nat et Int

-- Addition : proprietes fondamentales
#check @Nat.add_assoc    -- (a + b) + c = a + (b + c)
#check @Nat.add_comm     -- a + b = b + a
#check @Nat.add_zero     -- a + 0 = a
#check @Nat.zero_add     -- 0 + a = a

-- Multiplication : proprietes fondamentales  
#check @Nat.mul_assoc    -- (a * b) * c = a * (b * c)
#check @Nat.mul_comm     -- a * b = b * a
#check @Nat.mul_one      -- a * 1 = a
#check @Nat.one_mul      -- 1 * a = a

-- Distributivite
#check @Nat.left_distrib  -- a * (b + c) = a * b + a * c
#check @Nat.right_distrib -- (a + b) * c = a * c + b * c

/-!
## Algebre avec Mathlib

Avec `import Mathlib.Algebra.Group.Basic`, on accede aux proprietes generiques :

**Groupes :**
- `mul_one` : a * 1 = a
- `one_mul` : 1 * a = a  
- `mul_assoc` : (a * b) * c = a * (b * c)
- `mul_inv_cancel` : a * a⁻¹ = 1

**Anneaux :**
- `add_mul` : (a + b) * c = a * c + b * c
- `mul_add` : a * (b + c) = a * b + a * c
- `neg_add_cancel` : -a + a = 0

Ces theoremes s'appliquent a TOUS les types avec la structure adequate
(Int, Rat, Real, Complex, matrices, polynomes, etc.).
-/

-- Sans Mathlib, on a les proprietes specifiques aux types concrets
-- Voici quelques exemples avec Nat et Int

-- Addition : proprietes fondamentales
#check @Nat.add_assoc    -- (a + b) + c = a + (b + c)
──────▶  Nat.add_assoc : ∀ (n m k : Nat), n + m + k = n + (m + k)
#check @Nat.add_comm     -- a + b = b + a
──────▶  Nat.add_comm : ∀ (n m : Nat), n + m = m + n
#check @Nat.add_zero     -- a + 0 = a
──────▶  Nat.add_zero : ∀ (n : Nat), n + 0 = n
#check @Nat.zero_add     -- 0 + a = a
──────▶  Nat.zero_add : ∀ (n : Nat), 0 + n = n

-- Multiplication : proprietes fondamentales  
#check @Nat.mul_assoc    -- (a * b) * c = a * (b * c)
──────▶  Nat.mul_assoc : ∀ (n m k : Nat), n * m * k = n * (m * k)
#check @Nat.mul_comm     -- a * b = b * a
──────▶  Nat.mul_comm : ∀ (n m : Nat), n * m = m * n
#check @Nat.mul_one      -- a * 1 = a
──────▶  Nat.mul_one : ∀ (n : Nat), n * 1 = n
#check @Nat.one_mul      -- 1 * a = a
──────▶  Nat.one_mul : ∀ (n : Nat), 1 * n = n

-- Distributivite
#check @Nat.left_distrib  -- a * (b + c) = a * b + a * c
──────▶  Nat.left_distrib : ∀ (n m k : Nat), n * (m + k) = n * m + n * k
#check @Nat.right_distrib -- (a + b) * c = a * c + b * c
──────▶  Nat.right_distrib : ∀ (n m k : Nat), (n + m) * k = n * k + m * k
--% env 10

Raw input:
{"cmd": "/-!\n## Algebre avec Mathlib\n\nAvec `import Mathlib.Algebra.Group.Basic`, on accede aux proprietes generiques :\n\n**Groupes :**\n- `mul_one` : a * 1 = a\n- `one_mul` : 1 * a = a  \n- `mul_assoc` : (a * b) * c = a * (b * c)\n- `mul_inv_cancel` : a * a\u207b\u00b9 = 1\n\n**Anneaux :**\n- `add_mul` : (a + b) * c = a * c + b * c\n- `mul_add` : a * (b + c) = a * b + a * c\n- `neg_add_cancel` : -a + a = 0\n\nCes theoremes s'appliquent a TOUS les types avec la structure adequate\n(Int, Rat, Real, Complex, matrices, polynomes, etc.).\n-/\n\n-- Sans Mathlib, on a les proprietes specifiques aux types concrets\n-- Voici quelques exemples avec Nat et Int\n\n-- Addition : proprietes fondamentales\n#check @Nat.add_assoc    -- (a + b) + c = a + (b + c)\n#check @Nat.add_comm     -- a + b = b + a\n#check @Nat.add_zero     -- a + 0 = a\n#check @Nat.zero_add     -- 0 + a = a\n\n-- Multiplication : proprietes fondamentales  \n#check @Nat.mul_assoc    -- (a * b) * c = a * (b * c)\n#check @Nat.mul_comm     -- a * b = b * a\n#check @Nat.mul_one      -- a * 1 = a\n#check @Nat.one_mul      -- 1 * a = a\n\n-- Distributivite\n#check @Nat.left_distrib  -- a * (b + c) = a * b + a * c\n#check @Nat.right_distrib -- (a + b) * c = a * c + b * c", "env": 9}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 25, "column": 0},
   "endPos": {"line": 25, "column": 6},
   "data": "Nat.add_assoc : ∀ (n m k : Nat), n + m + k = n + (m + k)"},
  {"severity": "info",
   "pos": {"line": 26, "column": 0},
   "endPos": {"line": 26, "column": 6},
   "data": "Nat.add_comm : ∀ (n m : Nat), n + m = m + n"},
  {"severity": "info",
   "pos": {"line": 27, "column": 0},
   "endPos": {"line": 27, "column": 6},
   "data": "Nat.add_zero : ∀ (n : Nat), n + 0 = n"},
  {"severity": "info",
   "pos": {"line": 28, "column": 0},
   "endPos": {"line": 28, "column": 6},
   "data": "Nat.zero_add : ∀ (n : Nat), 0 + n = n"},
  {"severity": "info",
   "pos": {"line": 31, "column": 0},
   "endPos": {"line": 31, "column": 6},
   "data": "Nat.mul_assoc : ∀ (n m k : Nat), n * m * k = n * (m * k)"},
  {"severity": "info",
   "pos": {"line": 32, "column": 0},
   "endPos": {"line": 32, "column": 6},
   "da

### 4.3 Analyse

In [12]:
/-!
## Analyse avec Mathlib

Avec `import Mathlib.Analysis.Calculus.Deriv.Basic`, on accede a :

**Derivees :**
- `HasDerivAt` : Definition de la derivee
- `deriv_add` : (f + g)' = f' + g'
- `deriv_mul` : (f * g)' = f' * g + f * g' (Leibniz)
- `deriv_pow` : (x^n)' = n * x^(n-1)

**Limites :**
- `Filter.Tendsto` : Definition des limites
- `tendsto_add` : lim(f + g) = lim f + lim g
- `tendsto_mul` : lim(f * g) = lim f * lim g

**Integration :**
- `MeasureTheory.integral_add` : Linearite de l'integrale

L'analyse dans Mathlib utilise les filtres pour definir les limites
de maniere tres generale (limites en un point, a l'infini, etc.).
-/

-- L'analyse reelle necessite Mathlib
-- Voici un apercu des concepts disponibles dans Lean de base

-- Types numeriques de base
#check Nat    -- Nombres naturels
#check Int    -- Entiers relatifs
#check Float  -- Flottants (approximation, pas pour preuves formelles)

-- Ordre sur les nombres
#check @Nat.le_trans     -- a <= b -> b <= c -> a <= c
#check @Nat.lt_of_le_of_lt  -- a <= b -> b < c -> a < c

-- Division euclidienne
#check @Nat.div_add_mod  -- n = n / d * d + n % d

/-!
## Analyse avec Mathlib

Avec `import Mathlib.Analysis.Calculus.Deriv.Basic`, on accede a :

**Derivees :**
- `HasDerivAt` : Definition de la derivee
- `deriv_add` : (f + g)' = f' + g'
- `deriv_mul` : (f * g)' = f' * g + f * g' (Leibniz)
- `deriv_pow` : (x^n)' = n * x^(n-1)

**Limites :**
- `Filter.Tendsto` : Definition des limites
- `tendsto_add` : lim(f + g) = lim f + lim g
- `tendsto_mul` : lim(f * g) = lim f * lim g

**Integration :**
- `MeasureTheory.integral_add` : Linearite de l'integrale

L'analyse dans Mathlib utilise les filtres pour definir les limites
de maniere tres generale (limites en un point, a l'infini, etc.).
-/

-- L'analyse reelle necessite Mathlib
-- Voici un apercu des concepts disponibles dans Lean de base

-- Types numeriques de base
#check Nat    -- Nombres naturels
──────▶  Nat : Type
#check Int    -- Entiers relatifs
──────▶  Int : Type
#check Float  -- Flottants (approximation, pas pour preuves formelles)
──────▶  Float : Type

-- Ordre sur les nombres
#check @Nat.le_trans     -- a <= b -> b <= c -> a <= c
──────▶  @Nat.le_trans : ∀ {n m k : Nat}, n ≤ m → m ≤ k → n ≤ k
#check @Nat.lt_of_le_of_lt  -- a <= b -> b < c -> a < c
──────▶  @Nat.lt_of_le_of_lt : ∀ {n m k : Nat}, n ≤ m → m < k → n < k

-- Division euclidienne
#check @Nat.div_add_mod  -- n = n / d * d + n % d
──────▶  Nat.div_add_mod : ∀ (m n : Nat), n * (m / n) + m % n = m
--% env 11

Raw input:
{"cmd": "/-!\n## Analyse avec Mathlib\n\nAvec `import Mathlib.Analysis.Calculus.Deriv.Basic`, on accede a :\n\n**Derivees :**\n- `HasDerivAt` : Definition de la derivee\n- `deriv_add` : (f + g)' = f' + g'\n- `deriv_mul` : (f * g)' = f' * g + f * g' (Leibniz)\n- `deriv_pow` : (x^n)' = n * x^(n-1)\n\n**Limites :**\n- `Filter.Tendsto` : Definition des limites\n- `tendsto_add` : lim(f + g) = lim f + lim g\n- `tendsto_mul` : lim(f * g) = lim f * lim g\n\n**Integration :**\n- `MeasureTheory.integral_add` : Linearite de l'integrale\n\nL'analyse dans Mathlib utilise les filtres pour definir les limites\nde maniere tres generale (limites en un point, a l'infini, etc.).\n-/\n\n-- L'analyse reelle necessite Mathlib\n-- Voici un apercu des concepts disponibles dans Lean de base\n\n-- Types numeriques de base\n#check Nat    -- Nombres naturels\n#check Int    -- Entiers relatifs\n#check Float  -- Flottants (approximation, pas pour preuves formelles)\n\n-- Ordre sur les nombres\n#check @Nat.le_trans     -- a <= b -> b <= c -> a <= c\n#check @Nat.lt_of_le_of_lt  -- a <= b -> b < c -> a < c\n\n-- Division euclidienne\n#check @Nat.div_add_mod  -- n = n / d * d + n % d", "env": 10}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 28, "column": 0},
   "endPos": {"line": 28, "column": 6},
   "data": "Nat : Type"},
  {"severity": "info",
   "pos": {"line": 29, "column": 0},
   "endPos": {"line": 29, "column": 6},
   "data": "Int : Type"},
  {"severity": "info",
   "pos": {"line": 30, "column": 0},
   "endPos": {"line": 30, "column": 6},
   "data": "Float : Type"},
  {"severity": "info",
   "pos": {"line": 33, "column": 0},
   "endPos": {"line": 33, "column": 6},
   "data": "@Nat.le_trans : ∀ {n m k : Nat}, n ≤ m → m ≤ k → n ≤ k"},
  {"severity": "info",
   "pos": {"line": 34, "column": 0},
   "endPos": {"line": 34, "column": 6},
   "data": "@Nat.lt_of_le_of_lt : ∀ {n m k : Nat}, n ≤ m → m < k → n < k"},
  {"severity": "info",
   "pos": {"line": 37, "column": 0},
   "endPos": {"line": 37, "column": 6},
   "data": "Nat.div_add_mod : ∀ (m n : Nat), n * (m / n) + m % n = m"}],
 "env": 11}

## 5. Recherche et Navigation

### 5.1 Documentation en ligne

La documentation Mathlib est disponible sur : https://leanprover-community.github.io/mathlib4_docs/

### 5.2 Loogle : Recherche syntaxique

**Loogle** (https://loogle.lean-lang.org/) permet de chercher des theoremes par leur signature de type.

In [13]:
/-!
## Loogle : Recherche syntaxique

**Loogle** (https://loogle.lean-lang.org/) permet de chercher des theoremes 
par leur signature de type.

### Exemples de requetes Loogle :

| Requete | Recherche |
|---------|-----------|
| `?a + 0 = ?a` | Theoreme sur addition et zero |
| `?a + ?b = ?b + ?a` | Commutativite de l'addition |
| `Nat.Prime, Nat.Odd` | Theoremes impliquant Prime et Odd |

### Syntaxe :
- `?a` : Variable de type quelconque
- `_` : Argument dont on ne se soucie pas  
- `A -> B` : Fonction de A vers B
-/

-- Demonstration : chercher des theoremes par leur forme
-- Ces theoremes existent dans la bibliotheque standard de Lean

-- "?a + 0 = ?a" -> Nat.add_zero
#check @Nat.add_zero  -- forall n, n + 0 = n

-- "?a + ?b = ?b + ?a" -> Nat.add_comm
#check @Nat.add_comm  -- forall n m, n + m = m + n

-- "?a * 1 = ?a" -> Nat.mul_one
#check @Nat.mul_one   -- forall n, n * 1 = n

/-!
## Loogle : Recherche syntaxique

**Loogle** (https://loogle.lean-lang.org/) permet de chercher des theoremes 
par leur signature de type.

### Exemples de requetes Loogle :

| Requete | Recherche |
|---------|-----------|
| `?a + 0 = ?a` | Theoreme sur addition et zero |
| `?a + ?b = ?b + ?a` | Commutativite de l'addition |
| `Nat.Prime, Nat.Odd` | Theoremes impliquant Prime et Odd |

### Syntaxe :
- `?a` : Variable de type quelconque
- `_` : Argument dont on ne se soucie pas  
- `A -> B` : Fonction de A vers B
-/

-- Demonstration : chercher des theoremes par leur forme
-- Ces theoremes existent dans la bibliotheque standard de Lean

-- "?a + 0 = ?a" -> Nat.add_zero
#check @Nat.add_zero  -- forall n, n + 0 = n
──────▶  Nat.add_zero : ∀ (n : Nat), n + 0 = n

-- "?a + ?b = ?b + ?a" -> Nat.add_comm
#check @Nat.add_comm  -- forall n m, n + m = m + n
──────▶  Nat.add_comm : ∀ (n m : Nat), n + m = m + n

-- "?a * 1 = ?a" -> Nat.mul_one
#check @Nat.mul_one   -- forall n, n * 1 = n
──────▶  Nat.mul_one : ∀ (n : Nat), n * 1 = n
--% env 12

Raw input:
{"cmd": "/-!\n## Loogle : Recherche syntaxique\n\n**Loogle** (https://loogle.lean-lang.org/) permet de chercher des theoremes \npar leur signature de type.\n\n### Exemples de requetes Loogle :\n\n| Requete | Recherche |\n|---------|-----------|\n| `?a + 0 = ?a` | Theoreme sur addition et zero |\n| `?a + ?b = ?b + ?a` | Commutativite de l'addition |\n| `Nat.Prime, Nat.Odd` | Theoremes impliquant Prime et Odd |\n\n### Syntaxe :\n- `?a` : Variable de type quelconque\n- `_` : Argument dont on ne se soucie pas  \n- `A -> B` : Fonction de A vers B\n-/\n\n-- Demonstration : chercher des theoremes par leur forme\n-- Ces theoremes existent dans la bibliotheque standard de Lean\n\n-- \"?a + 0 = ?a\" -> Nat.add_zero\n#check @Nat.add_zero  -- forall n, n + 0 = n\n\n-- \"?a + ?b = ?b + ?a\" -> Nat.add_comm\n#check @Nat.add_comm  -- forall n m, n + m = m + n\n\n-- \"?a * 1 = ?a\" -> Nat.mul_one\n#check @Nat.mul_one   -- forall n, n * 1 = n", "env": 11}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 25, "column": 0},
   "endPos": {"line": 25, "column": 6},
   "data": "Nat.add_zero : ∀ (n : Nat), n + 0 = n"},
  {"severity": "info",
   "pos": {"line": 28, "column": 0},
   "endPos": {"line": 28, "column": 6},
   "data": "Nat.add_comm : ∀ (n m : Nat), n + m = m + n"},
  {"severity": "info",
   "pos": {"line": 31, "column": 0},
   "endPos": {"line": 31, "column": 6},
   "data": "Nat.mul_one : ∀ (n : Nat), n * 1 = n"}],
 "env": 12}

### 5.3 Moogle : Recherche semantique

**Moogle** (https://www.moogle.ai/) utilise des LLMs pour la recherche semantique dans Mathlib - on peut chercher en langage naturel.

In [14]:
/-!
## Moogle : Recherche semantique

**Moogle** (https://www.moogle.ai/) utilise des LLMs pour la recherche 
semantique dans Mathlib - on peut chercher en langage naturel.

### Exemples de requetes Moogle :

| Requete (langage naturel) | Resultat |
|---------------------------|----------|
| "derivative of exponential function" | `Real.deriv_exp : deriv exp x = exp x` |
| "infinitely many prime numbers" | `Nat.exists_infinite_primes` |
| "triangle inequality" | `norm_add_le : ‖a + b‖ <= ‖a‖ + ‖b‖` |
| "fundamental theorem of calculus" | `intervalIntegral.integral_eq_sub_of_hasDerivAt` |

### Avantages de Moogle :
- Recherche en langage naturel
- Comprend les synonymes mathematiques
- Suggere des theoremes connexes
-/

-- Demonstration de la recherche : lemmes utiles sur les listes
#check @List.length_append  -- length (l1 ++ l2) = length l1 + length l2
#check @List.length_reverse -- length (reverse l) = length l
#check @List.reverse_reverse -- reverse (reverse l) = l

/-!
## Moogle : Recherche semantique

**Moogle** (https://www.moogle.ai/) utilise des LLMs pour la recherche 
semantique dans Mathlib - on peut chercher en langage naturel.

### Exemples de requetes Moogle :

| Requete (langage naturel) | Resultat |
|---------------------------|----------|
| "derivative of exponential function" | `Real.deriv_exp : deriv exp x = exp x` |
| "infinitely many prime numbers" | `Nat.exists_infinite_primes` |
| "triangle inequality" | `norm_add_le : ‖a + b‖ <= ‖a‖ + ‖b‖` |
| "fundamental theorem of calculus" | `intervalIntegral.integral_eq_sub_of_hasDerivAt` |

### Avantages de Moogle :
- Recherche en langage naturel
- Comprend les synonymes mathematiques
- Suggere des theoremes connexes
-/

-- Demonstration de la recherche : lemmes utiles sur les listes
#check @List.length_append  -- length (l1 ++ l2) = length l1 + length l2
──────▶  @List.length_append : ∀ {α : Type u_1} (as bs : List α), (as ++ bs).length = as.length + bs.length
#check @List.length_reverse -- length (reverse l) = length l
──────▶  @List.length_reverse : ∀ {α : Type u_1} (as : List α), as.reverse.length = as.length
#check @List.reverse_reverse -- reverse (reverse l) = l
──────▶  @List.reverse_reverse : ∀ {α : Type u_1} (as : List α), as.reverse.reverse = as
--% env 13

Raw input:
{"cmd": "/-!\n## Moogle : Recherche semantique\n\n**Moogle** (https://www.moogle.ai/) utilise des LLMs pour la recherche \nsemantique dans Mathlib - on peut chercher en langage naturel.\n\n### Exemples de requetes Moogle :\n\n| Requete (langage naturel) | Resultat |\n|---------------------------|----------|\n| \"derivative of exponential function\" | `Real.deriv_exp : deriv exp x = exp x` |\n| \"infinitely many prime numbers\" | `Nat.exists_infinite_primes` |\n| \"triangle inequality\" | `norm_add_le : \u2016a + b\u2016 <= \u2016a\u2016 + \u2016b\u2016` |\n| \"fundamental theorem of calculus\" | `intervalIntegral.integral_eq_sub_of_hasDerivAt` |\n\n### Avantages de Moogle :\n- Recherche en langage naturel\n- Comprend les synonymes mathematiques\n- Suggere des theoremes connexes\n-/\n\n-- Demonstration de la recherche : lemmes utiles sur les listes\n#check @List.length_append  -- length (l1 ++ l2) = length l1 + length l2\n#check @List.length_reverse -- length (reverse l) = length l\n#check @List.reverse_reverse -- reverse (reverse l) = l", "env": 12}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 23, "column": 0},
   "endPos": {"line": 23, "column": 6},
   "data":
   "@List.length_append : ∀ {α : Type u_1} (as bs : List α), (as ++ bs).length = as.length + bs.length"},
  {"severity": "info",
   "pos": {"line": 24, "column": 0},
   "endPos": {"line": 24, "column": 6},
   "data":
   "@List.length_reverse : ∀ {α : Type u_1} (as : List α), as.reverse.length = as.length"},
  {"severity": "info",
   "pos": {"line": 25, "column": 0},
   "endPos": {"line": 25, "column": 6},
   "data":
   "@List.reverse_reverse : ∀ {α : Type u_1} (as : List α), as.reverse.reverse = as"}],
 "env": 13}

### 5.4 Commandes de recherche dans Lean

In [15]:
/-!
## Commandes de recherche dans Lean

Lean (avec Mathlib) fournit des tactiques de recherche interactives :

| Commande | Usage |
|----------|-------|
| `exact?` | Cherche une preuve exacte dans le contexte et la bibliotheque |
| `apply?` | Cherche un lemme applicable au but courant |
| `rw?` | Suggere des reecritures possibles |
| `simp?` | Montre les regles simp utilisees |

### Exemples d'utilisation :

```lean
-- exact? cherche une preuve exacte
example (n : Nat) : n + 0 = n := by exact?
-- Resultat : exact Nat.add_zero n

-- apply? cherche un lemme a appliquer
example (n m : Nat) : n <= n + m := by apply?
-- Resultat : apply Nat.le_add_right

-- rw? suggere des reecritures
example (n : Nat) : 0 + n = n := by rw?
-- Resultat : rw [Nat.zero_add]
```
-/

-- Sans les tactiques de recherche Mathlib, on utilise les lemmes directement
-- Voici comment on applique ces lemmes manuellement

-- Au lieu de exact?, on utilise le lemme explicitement
example (n : Nat) : n + 0 = n := Nat.add_zero n

-- Au lieu de apply?, on connait le lemme a appliquer
example (n m : Nat) : n <= n + m := Nat.le_add_right n m

-- Au lieu de rw?, on specifie la reecriture
example (n : Nat) : 0 + n = n := by rw [Nat.zero_add]

/-!
## Commandes de recherche dans Lean

Lean (avec Mathlib) fournit des tactiques de recherche interactives :

| Commande | Usage |
|----------|-------|
| `exact?` | Cherche une preuve exacte dans le contexte et la bibliotheque |
| `apply?` | Cherche un lemme applicable au but courant |
| `rw?` | Suggere des reecritures possibles |
| `simp?` | Montre les regles simp utilisees |

### Exemples d'utilisation :

```lean
-- exact? cherche une preuve exacte
example (n : Nat) : n + 0 = n := by exact?
-- Resultat : exact Nat.add_zero n

-- apply? cherche un lemme a appliquer
example (n m : Nat) : n <= n + m := by apply?
-- Resultat : apply Nat.le_add_right

-- rw? suggere des reecritures
example (n : Nat) : 0 + n = n := by rw?
-- Resultat : rw [Nat.zero_add]
```
-/

-- Sans les tactiques de recherche Mathlib, on utilise les lemmes directement
-- Voici comment on applique ces lemmes manuellement

-- Au lieu de exact?, on utilise le lemme explicitement
example (n : Nat) : n + 0 = n := Nat.add_zero n

-- Au lieu de apply?, on connait le lemme a appliquer
example (n m : Nat) : n <= n + m := Nat.le_add_right n m

-- Au lieu de rw?, on specifie la reecriture
example (n : Nat) : 0 + n = n := by rw [Nat.zero_add]
--% env 14

Raw input:
{"cmd": "/-!\n## Commandes de recherche dans Lean\n\nLean (avec Mathlib) fournit des tactiques de recherche interactives :\n\n| Commande | Usage |\n|----------|-------|\n| `exact?` | Cherche une preuve exacte dans le contexte et la bibliotheque |\n| `apply?` | Cherche un lemme applicable au but courant |\n| `rw?` | Suggere des reecritures possibles |\n| `simp?` | Montre les regles simp utilisees |\n\n### Exemples d'utilisation :\n\n```lean\n-- exact? cherche une preuve exacte\nexample (n : Nat) : n + 0 = n := by exact?\n-- Resultat : exact Nat.add_zero n\n\n-- apply? cherche un lemme a appliquer\nexample (n m : Nat) : n <= n + m := by apply?\n-- Resultat : apply Nat.le_add_right\n\n-- rw? suggere des reecritures\nexample (n : Nat) : 0 + n = n := by rw?\n-- Resultat : rw [Nat.zero_add]\n```\n-/\n\n-- Sans les tactiques de recherche Mathlib, on utilise les lemmes directement\n-- Voici comment on applique ces lemmes manuellement\n\n-- Au lieu de exact?, on utilise le lemme explicitement\nexample (n : Nat) : n + 0 = n := Nat.add_zero n\n\n-- Au lieu de apply?, on connait le lemme a appliquer\nexample (n m : Nat) : n <= n + m := Nat.le_add_right n m\n\n-- Au lieu de rw?, on specifie la reecriture\nexample (n : Nat) : 0 + n = n := by rw [Nat.zero_add]", "env": 13}
Raw output:
{"env": 14}

## 6. Cas d'Usage Avances

### 6.1 Terry Tao et la theorie analytique des nombres

En janvier 2026, **Terence Tao** (medaille Fields 2006) a utilise Mathlib4 pour formaliser des resultats en theorie analytique des nombres. Il a notamment contribue a :

- Formalisation de lemmes sur les fonctions L de Dirichlet
- Verification de bornes asymptotiques pour la fonction pi(x)
- Travail vers une preuve formelle complete du theoreme des nombres premiers

Cet evenement marque un tournant : un mathematicien de premier plan utilise la verification formelle comme outil de recherche quotidien.

### 6.2 Theoremes majeurs formalises dans Mathlib

| Theoreme | Domaine | Auteur original |
|----------|---------|-----------------|
| Dernier theoreme de Fermat | Theorie des nombres | Fermat/Wiles |
| Theoreme des nombres premiers | Analyse | Hadamard/de la Vallee Poussin |
| Theoreme de Bolzano-Weierstrass | Analyse | Bolzano |
| Lemme de Zorn | Theorie des ensembles | Zorn |
| Theoreme de Hahn-Banach | Analyse fonctionnelle | Hahn/Banach |
| Theoreme de Stone-Weierstrass | Approximation | Stone/Weierstrass |

### 6.3 Lean4Lean : Le bootstrap (POPL 2026)

**Lean4Lean** est un projet ambitieux qui vise a reimplementer Lean 4 en Lean 4 lui-meme. Presente a POPL 2026, il demontre :

- **Auto-hebergement** : Lean peut se compiler lui-meme
- **Verification** : Le compilateur est prouve correct
- **Confiance accrue** : Reduit la base de code "de confiance"

### 6.4 Impact sur la recherche mathematique

**Avant la formalisation** :
- Revue par pairs humaine, erreurs possibles
- Temps de verification : mois/annees
- Confiance : depend de la reputation

**Avec Mathlib/Lean** :
- Verification automatique, certitude mathematique
- Temps : minutes/heures pour verifier
- Confiance : absolue si la preuve compile

## 7. Exercices

### Exercice 1 : Utiliser omega

In [16]:
-- Prouver avec omega
theorem ex1 (a b c : Nat) (h1 : a + b = c) (h2 : c < 10) : a < 10 := by
  sorry

-- Prouver avec omega
theorem ex1 (a b c : Nat) (h1 : a + b = c) (h2 : c < 10) : a < 10 := by
        ───▶ 🟨 declaration uses 'sorry'
  sorry
--% env 15
--% prove 0

Raw input:
{"cmd": "-- Prouver avec omega\ntheorem ex1 (a b c : Nat) (h1 : a + b = c) (h2 : c < 10) : a < 10 := by\n  sorry", "env": 14}
Raw output:
{"sorries":
 [{"proofState": 0,
   "pos": {"line": 3, "column": 2},
   "goal": "a b c : Nat\nh1 : a + b = c\nh2 : c < 10\n⊢ a < 10",
   "endPos": {"line": 3, "column": 7}}],
 "messages":
 [{"severity": "warning",
   "pos": {"line": 2, "column": 8},
   "endPos": {"line": 2, "column": 11},
   "data": "declaration uses 'sorry'"}],
 "env": 15}

### Exercice 2 : Calculer avec rfl/decide

In [17]:
-- Prouver ces calculs
theorem ex2a : 7 * 8 = 56 := by sorry
theorem ex2b : 100 > 50 := by sorry

-- Prouver ces calculs
theorem ex2a : 7 * 8 = 56 := by sorry
        ────▶ 🟨 declaration uses 'sorry'
theorem ex2b : 100 > 50 := by sorry
        ────▶ 🟨 declaration uses 'sorry'
--% env 16
--% prove 2

Raw input:
{"cmd": "-- Prouver ces calculs\ntheorem ex2a : 7 * 8 = 56 := by sorry\ntheorem ex2b : 100 > 50 := by sorry", "env": 15}
Raw output:
{"sorries":
 [{"proofState": 1,
   "pos": {"line": 2, "column": 32},
   "goal": "⊢ 7 * 8 = 56",
   "endPos": {"line": 2, "column": 37}},
  {"proofState": 2,
   "pos": {"line": 3, "column": 30},
   "goal": "⊢ 100 > 50",
   "endPos": {"line": 3, "column": 35}}],
 "messages":
 [{"severity": "warning",
   "pos": {"line": 2, "column": 8},
   "endPos": {"line": 2, "column": 12},
   "data": "declaration uses 'sorry'"},
  {"severity": "warning",
   "pos": {"line": 3, "column": 8},
   "endPos": {"line": 3, "column": 12},
   "data": "declaration uses 'sorry'"}],
 "env": 16}

### Exercice 3 : Preuve manuelle style Mathlib

In [18]:
-- Prouver l'associativite de l'addition sur Nat
-- (sans utiliser directement Nat.add_assoc)
theorem ex3 (a b c : Nat) : (a + b) + c = a + (b + c) := by
  sorry

-- Prouver l'associativite de l'addition sur Nat
-- (sans utiliser directement Nat.add_assoc)
theorem ex3 (a b c : Nat) : (a + b) + c = a + (b + c) := by
        ───▶ 🟨 declaration uses 'sorry'
  sorry
--% env 17
--% prove 3

Raw input:
{"cmd": "-- Prouver l'associativite de l'addition sur Nat\n-- (sans utiliser directement Nat.add_assoc)\ntheorem ex3 (a b c : Nat) : (a + b) + c = a + (b + c) := by\n  sorry", "env": 16}
Raw output:
{"sorries":
 [{"proofState": 3,
   "pos": {"line": 4, "column": 2},
   "goal": "a b c : Nat\n⊢ a + b + c = a + (b + c)",
   "endPos": {"line": 4, "column": 7}}],
 "messages":
 [{"severity": "warning",
   "pos": {"line": 3, "column": 8},
   "endPos": {"line": 3, "column": 11},
   "data": "declaration uses 'sorry'"}],
 "env": 17}

## Solutions

In [19]:
-- Solution exercice 1
theorem ex1_sol (a b c : Nat) (h1 : a + b = c) (h2 : c < 10) : a < 10 := by
  omega

-- Solution exercice 2
theorem ex2a_sol : 7 * 8 = 56 := by rfl
theorem ex2b_sol : 100 > 50 := by decide

-- Solution exercice 3
-- On utilise directement le lemme de la bibliotheque standard
theorem ex3_sol (a b c : Nat) : (a + b) + c = a + (b + c) :=
  Nat.add_assoc a b c

-- Solution exercice 1
theorem ex1_sol (a b c : Nat) (h1 : a + b = c) (h2 : c < 10) : a < 10 := by
  omega

-- Solution exercice 2
theorem ex2a_sol : 7 * 8 = 56 := by rfl
theorem ex2b_sol : 100 > 50 := by decide

-- Solution exercice 3
-- On utilise directement le lemme de la bibliotheque standard
theorem ex3_sol (a b c : Nat) : (a + b) + c = a + (b + c) :=
  Nat.add_assoc a b c
--% env 18

Raw input:
{"cmd": "-- Solution exercice 1\ntheorem ex1_sol (a b c : Nat) (h1 : a + b = c) (h2 : c < 10) : a < 10 := by\n  omega\n\n-- Solution exercice 2\ntheorem ex2a_sol : 7 * 8 = 56 := by rfl\ntheorem ex2b_sol : 100 > 50 := by decide\n\n-- Solution exercice 3\n-- On utilise directement le lemme de la bibliotheque standard\ntheorem ex3_sol (a b c : Nat) : (a + b) + c = a + (b + c) :=\n  Nat.add_assoc a b c", "env": 17}
Raw output:
{"env": 18}

## Resume

### Tactiques Mathlib essentielles

| Tactique | Domaine | Usage | Disponibilite |
|----------|---------|-------|---------------|
| `ring` | Algebre | Egalites polynomiales | Mathlib |
| `linarith` | Arithmetique | Inegalites lineaires | Mathlib |
| `omega` | Presburger | Arithmetique avec +, -, < | Lean de base |
| `norm_num` | Calcul | Expressions numeriques | Mathlib |
| `field_simp` | Corps | Fractions et divisions | Mathlib |
| `polyrith` | Polynomes | Certificats via Sage | Mathlib |
| `simp` | General | Simplification automatique | Lean de base |
| `exact?` | Recherche | Trouver une preuve exacte | Mathlib |
| `apply?` | Recherche | Trouver un lemme applicable | Mathlib |
| `decide` | Decision | Propositions decidables | Lean de base |
| `rfl` | Egalite | Reflexivite definitionnelle | Lean de base |

### Structure Mathlib4

| Namespace | Contenu principal |
|-----------|-------------------|
| `Mathlib.Data` | List, Set, Finset, Multiset, Nat, Int, Rat |
| `Mathlib.Algebra` | Group, Ring, Field, Module, LinearMap |
| `Mathlib.Order` | Lattice, Order, Filter |
| `Mathlib.Topology` | TopologicalSpace, Compactness, Continuity |
| `Mathlib.Analysis` | Limits, Derivatives, Integrals, Measures |
| `Mathlib.NumberTheory` | Prime, Divisibility, Modular arithmetic |
| `Mathlib.Tactic` | ring, linarith, omega, norm_num, etc. |

### Ressources

| Ressource | URL | Usage |
|-----------|-----|-------|
| Documentation officielle | https://leanprover-community.github.io/mathlib4_docs/ | Reference complete |
| Loogle | https://loogle.lean-lang.org/ | Recherche par signature de type |
| Moogle | https://www.moogle.ai/ | Recherche en langage naturel |
| Repository GitHub | https://github.com/leanprover-community/mathlib4 | Code source |
| Zulip Chat | https://leanprover.zulipchat.com/ | Communaute et support |
| Mathematics in Lean | https://leanprover-community.github.io/mathematics_in_lean/ | Tutoriel approfondi |

### Commandes de recherche

```lean
-- exact? : trouve une preuve exacte
-- apply? : trouve un lemme a appliquer  
-- rw?    : suggere des reecritures
-- simp?  : montre les regles simp utilisees
```

### Prochaine etape

Dans le notebook **Lean-7-LLM-Integration**, nous verrons comment les **Large Language Models** peuvent assister la construction de preuves Lean, avec des outils comme **LeanCopilot** (NeurIPS 2025), **LeanProgress** (TMLR 2025), et les percees recentes d'**AlphaProof** (Nature 2025).

---

*Notebook base sur l'etat de l'art Mathlib4 v4.27.0-rc1 (janvier 2026)*
*Ressources : leanprover-community.github.io, Terry Tao blog, Lean4Lean (POPL 2026)*

---

**Navigation** : [← Lean-5-Tactics](Lean-5-Tactics.ipynb) | [Index](Lean-1-Setup.ipynb) | [Lean-7-LLM-Integration →](Lean-7-LLM-Integration.ipynb)